In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt


In [2]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')


In [3]:

train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)


In [4]:
train.shape

(1458, 81)

In [5]:
test.shape

(1459, 80)

In [6]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
test['SalePrice']=''

In [8]:
test.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,
1458,2919,60,RL,74.0,9627,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,11,2006,WD,Normal,


In [9]:
train_test=pd.concat([train, test])

In [10]:
train_test.shape

(2917, 81)

In [11]:
train_test.isna().sum()

Id                 0
MSSubClass         0
MSZoning           4
LotFrontage      486
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           1
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [12]:
train_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2917 entries, 0 to 1458
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             2917 non-null   int64  
 1   MSSubClass     2917 non-null   int64  
 2   MSZoning       2913 non-null   object 
 3   LotFrontage    2431 non-null   float64
 4   LotArea        2917 non-null   int64  
 5   Street         2917 non-null   object 
 6   Alley          198 non-null    object 
 7   LotShape       2917 non-null   object 
 8   LandContour    2917 non-null   object 
 9   Utilities      2915 non-null   object 
 10  LotConfig      2917 non-null   object 
 11  LandSlope      2917 non-null   object 
 12  Neighborhood   2917 non-null   object 
 13  Condition1     2917 non-null   object 
 14  Condition2     2917 non-null   object 
 15  BldgType       2917 non-null   object 
 16  HouseStyle     2917 non-null   object 
 17  OverallQual    2917 non-null   int64  
 18  OverallC

In [13]:
train_test['MSSubClass'].describe()

count    2917.000000
mean       57.135756
std        42.532140
min        20.000000
25%        20.000000
50%        50.000000
75%        70.000000
max       190.000000
Name: MSSubClass, dtype: float64

In [14]:
train_test['PoolQC']=train_test['PoolQC'].fillna("None")
train_test['Fence']=train_test['Fence'].fillna("None")
train_test['MiscFeature']=train_test['MiscFeature'].fillna("None")
train_test['Alley']=train_test['Alley'].fillna("None")
train_test['FireplaceQu']=train_test['FireplaceQu'].fillna("None")

In [15]:
train_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2917 entries, 0 to 1458
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             2917 non-null   int64  
 1   MSSubClass     2917 non-null   int64  
 2   MSZoning       2913 non-null   object 
 3   LotFrontage    2431 non-null   float64
 4   LotArea        2917 non-null   int64  
 5   Street         2917 non-null   object 
 6   Alley          2917 non-null   object 
 7   LotShape       2917 non-null   object 
 8   LandContour    2917 non-null   object 
 9   Utilities      2915 non-null   object 
 10  LotConfig      2917 non-null   object 
 11  LandSlope      2917 non-null   object 
 12  Neighborhood   2917 non-null   object 
 13  Condition1     2917 non-null   object 
 14  Condition2     2917 non-null   object 
 15  BldgType       2917 non-null   object 
 16  HouseStyle     2917 non-null   object 
 17  OverallQual    2917 non-null   int64  
 18  OverallC

In [16]:
train_test['LotFrontage']=train_test.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.mean()))

In [17]:
train_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2917 entries, 0 to 1458
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             2917 non-null   int64  
 1   MSSubClass     2917 non-null   int64  
 2   MSZoning       2913 non-null   object 
 3   LotFrontage    2917 non-null   float64
 4   LotArea        2917 non-null   int64  
 5   Street         2917 non-null   object 
 6   Alley          2917 non-null   object 
 7   LotShape       2917 non-null   object 
 8   LandContour    2917 non-null   object 
 9   Utilities      2915 non-null   object 
 10  LotConfig      2917 non-null   object 
 11  LandSlope      2917 non-null   object 
 12  Neighborhood   2917 non-null   object 
 13  Condition1     2917 non-null   object 
 14  Condition2     2917 non-null   object 
 15  BldgType       2917 non-null   object 
 16  HouseStyle     2917 non-null   object 
 17  OverallQual    2917 non-null   int64  
 18  OverallC

In [18]:
train_test['MasVnrArea']=train_test['MasVnrArea'].fillna(0)
train_test['MasVnrType']=train_test['MasVnrArea'].fillna('none')

In [19]:
train_test['has_garage']=train_test['GarageArea'].apply(lambda x:1 if x>0 else 0)
train_test['has_basement']=train_test['TotalBsmtSF'].apply(lambda x:1 if x>0 else 0)


In [20]:
train_test.shape

(2917, 83)

In [21]:
train_test=train_test.replace({'MSSubClass': {20:'SC20', 30:'SC30', 40:'SC40', 45:'SC45', 50:'SC50', 60:'SC60', 70:'SC70', 75:'SC75', 80:'SC80', 85:'SC85', 90:'SC90', 120:'SC120', 150:'SC150', 160:'SC160', 180:'SC180', 190:'SC190'}
                               , 'MoSold': {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}})

In [22]:
train_test.shape

(2917, 83)

In [23]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [24]:
train=train_test.iloc[:1458,:]
test=train_test.iloc[1458: , :].drop('SalePrice', axis=1)
train.shape, test.shape

((1458, 83), (1459, 82))

In [25]:
train=train.dropna()

In [26]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,has_garage,has_basement
0,1461,SC20,RH,80.0,11622,Pave,None,Reg,Lvl,AllPub,...,None,MnPrv,None,0,Jun,2010,WD,Normal,1,1
1,1462,SC20,RL,81.0,14267,Pave,None,IR1,Lvl,AllPub,...,None,None,Gar2,12500,Jun,2010,WD,Normal,1,1
2,1463,SC60,RL,74.0,13830,Pave,None,IR1,Lvl,AllPub,...,None,MnPrv,None,0,Mar,2010,WD,Normal,1,1
3,1464,SC60,RL,78.0,9978,Pave,None,IR1,Lvl,AllPub,...,None,None,None,0,Jun,2010,WD,Normal,1,1
4,1465,SC120,RL,43.0,5005,Pave,None,IR1,HLS,AllPub,...,None,None,None,0,Jan,2010,WD,Normal,1,1


In [27]:
y=train['SalePrice'].astype({'SalePrice':'float64'})
y=np.log1p(y)
y
train=train.drop('SalePrice', axis=1)

In [28]:

train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,has_garage,has_basement
0,1,SC60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,None,None,None,0,Feb,2008,WD,Normal,1,1
1,2,SC20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,None,None,None,0,May,2007,WD,Normal,1,1
2,3,SC60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,None,None,None,0,Sep,2008,WD,Normal,1,1
3,4,SC70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,None,None,None,0,Feb,2006,WD,Abnorml,1,1
4,5,SC60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,None,None,None,0,Dec,2008,WD,Normal,1,1


In [29]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 1458
Data columns (total 82 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   object 
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1459 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          1459 non-null   object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [30]:
for col in test.columns:
    if test[col].isna().sum() > 0:
        if test[col].dtypes == np.int64 or test[col].dtypes == np.float64:
            test[col]=test[col].fillna(0)
        elif test[col].dtypes== np.object:
            test[col]=test[col].fillna('None')
test.isna().sum().sum()           

0

In [31]:
train.shape, test.shape

((1344, 82), (1459, 82))

In [32]:
tt=pd.concat([train, test])
tt=pd.get_dummies(tt)
tt.shape

(2803, 333)

In [33]:
train=tt.iloc[:len(y), : ]
test=tt.iloc[len(y):, : ]

In [34]:
t

NameError: name 't' is not defined

In [35]:
y.head()

0    12.247699
1    12.109016
2    12.317171
3    11.849405
4    12.429220
Name: SalePrice, dtype: float64

In [36]:
from sklearn.linear_model import Ridge
clf=Ridge(alpha=10).fit(train, y)
preds=clf.predict(test)
preds=np.expm1(preds).astype(int)
preds



array([118491, 158636, 178948, ..., 172063, 125982, 222188])

In [37]:
sub=pd.DataFrame()
sub['Id']=test['Id']
sub['SalePrice']=preds
sub.to_csv('submission.csv', index=False)